In [1]:
!pip install pyspark 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 37 kB/s 
     |████████████████████████████████| 199 kB 46.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845514 sha256=45549c162778e2e301085294b516326b0f48707da4aa8791a38eedd7188c11c4
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [2]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

spark = SparkSession\
        .builder\
        .master('local[*]')\
        .getOrCreate()
sc = spark.sparkContext

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#change the directory
%cd /content/drive/My Drive/Colab Notebooks/big-data-Colab/
#pwd
!ls

/content/drive/My Drive/Colab Notebooks/big-data-Colab
 2.DataCopy.ipynb		 JosephNikhilReddy_MT_Q2_A.txt
 2.WordCount.ipynb		 JosephNikhilReddy_MT_Q2_B.txt
 4.MR-DataJoin.ipynb		 JosephNikhilReddy_MT_Q3_A.txt
 4.MR-WordCount.ipynb		 JosephNikhilReddy_MT_Q3_B.txt
 5.MR-DataJoin2.ipynb		 make_join2data.py
 6.MR-WordCountReducer.ipynb	'Midterm Section2 Q1.ipynb'
 7.Spark-WordCount.ipynb	'Midterm Section2 Q2.ipynb'
 8.Spark-SQL.ipynb		'Midterm Section2 Q3.ipynb'
 8.Spark-Streaming.ipynb	 MT_Q1_make_data.py
 ad-clicks.csv.gz		 MT_Q2_make_data.py
 BigData.ipynb			 MT_Q3_make_data.py
 buy-clicks.csv.gz		 out
 daily_weather.csv		 out0
 game-clicks.csv.gz		 out2
 hadoop-3.3.4			 out3
 hadoop-3.3.4.tar.gz		 outDataJoin
 join1_FileA.txt		 outDataJoin2
 join1_FileB.txt		 Q1out
 join1_mapper.py		 Q2out
 join1_mapper_Q2.py		 SparkOutPut
 join1_reducer.py		 testfile1
 join1_reducer_Q2.py		 testfile2
 join2_genchanA.txt		 Untitled0.ipynb
 join2_genchanB.txt		 wordcount_mapper.py
 join2_genchanC.txt		 

In [5]:
from pyspark.sql import SQLContext
sqlsc = SQLContext(sc)

df =sqlsc.read.load('/content/drive/My Drive/Colab Notebooks/big-data-Colab/daily_weather.csv',
                          format='com.databricks.spark.csv',
                          header = 'true',
                          inferSchema='true')

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:114: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning,


In [6]:
df.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
number,1095,547.0,316.24357700987383,0,1094
air_pressure_9am,1092,918.8825513138094,3.184161180386833,907.9900000000024,929.3200000000012
air_temp_9am,1090,64.93300141287072,11.175514003175877,36.752000000000685,98.90599999999992
avg_wind_direction_9am,1091,142.2355107005759,69.13785928889189,15.500000000000046,343.4
avg_wind_speed_9am,1092,5.50828424225493,4.5528134655317185,0.69345139999974,23.554978199999763
max_wind_direction_9am,1092,148.95351796516923,67.23801294602953,28.89999999999991,312.19999999999993
max_wind_speed_9am,1091,7.019513529175272,5.598209170780958,1.1855782000000479,29.84077959999996
rain_accumulation_9am,1089,0.20307895225211126,1.5939521253574893,0.0,24.01999999999907
rain_duration_9am,1092,294.1080522756142,1598.0787786601481,0.0,17704.0


In [7]:
df.describe(['air_temp_9am']).show()

+-------+------------------+
|summary|      air_temp_9am|
+-------+------------------+
|  count|              1090|
|   mean| 64.93300141287072|
| stddev|11.175514003175877|
|    min|36.752000000000685|
|    max| 98.90599999999992|
+-------+------------------+



In [8]:
df.count()

1095

In [9]:
removeALLDF= df.na.drop()

In [10]:
removeALLDF.describe(['air_temp_9am']).show()

+-------+------------------+
|summary|      air_temp_9am|
+-------+------------------+
|  count|              1064|
|   mean| 65.02260949558733|
| stddev|11.168033449415704|
|    min|36.752000000000685|
|    max| 98.90599999999992|
+-------+------------------+



In [11]:
removeALLDF.count()

1064

In [12]:
from pyspark.sql.functions import avg
imputeDF = df

In [13]:
for x in imputeDF.columns:
    meanValue = removeALLDF.agg(avg(x)).first()[0]
    print(x, meanValue)
    imputeDF= imputeDF.na.fill(meanValue, [x])
    

number 545.0018796992481
air_pressure_9am 918.9031798641051
air_temp_9am 65.02260949558733
avg_wind_direction_9am 142.30675564934037
avg_wind_speed_9am 5.48579305071369
max_wind_direction_9am 148.48042413321315
max_wind_speed_9am 6.999713658875691
rain_accumulation_9am 0.18202347650615522
rain_duration_9am 266.3936973996037
relative_humidity_9am 34.07743985327709
relative_humidity_3pm 35.14838093290533


In [14]:
df.describe(['air_temp_9am']).show()
imputeDF.describe(['air_temp_9am']).show()

+-------+------------------+
|summary|      air_temp_9am|
+-------+------------------+
|  count|              1090|
|   mean| 64.93300141287072|
| stddev|11.175514003175877|
|    min|36.752000000000685|
|    max| 98.90599999999992|
+-------+------------------+

+-------+------------------+
|summary|      air_temp_9am|
+-------+------------------+
|  count|              1095|
|   mean| 64.93341058219818|
| stddev| 11.14994819992023|
|    min|36.752000000000685|
|    max| 98.90599999999992|
+-------+------------------+



In [15]:
imputeDF.count()

1095